In [3]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import t

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent
os.chdir(ROOT)

DATA_DIR = Path("synthetic-data") / "affinity"
OUT_DIR = Path("outputs") / "affinity"
OUT_DIR.mkdir(parents=True, exist_ok=True)

def four_param_logistic(conc, A, B, C, D):
    return D + (A - D) / (1 + (conc / C) ** B)

outputs = []

for file_path in sorted(DATA_DIR.glob("*.csv")):
    df = pd.read_csv(file_path)

    if not {"X", "Y", "Condition"}.issubset(df.columns):
        continue

    name = file_path.stem
    parts = name.split("_")
    compound_name = parts[1] if len(parts) > 1 else name

    for condition in df["Condition"].dropna().unique():
        subset = df[df["Condition"] == condition]
        x = subset["X"].to_numpy(dtype=float)
        y = subset["Y"].to_numpy(dtype=float)

        if len(x) < 11:
            continue

        try:
            A_init = float(np.nanmax(y))
            D_init = float(np.nanmin(y))
            C_init = float(np.nanmedian(x))
            B_init = 1.0
            p0 = [A_init, B_init, C_init, D_init]

            popt, pcov = curve_fit(
                four_param_logistic,
                x,
                y,
                p0=p0,
                bounds=(0, np.inf),
                maxfev=10000
            )

            A, B, C, D = popt
            dof = max(0, len(x) - len(popt))
            tval = t.ppf(0.975, dof) if dof > 0 else np.nan
            perr = np.sqrt(np.diag(pcov)) if pcov is not None else np.full(4, np.nan)
            ci = perr * tval if np.isfinite(tval) else np.full(4, np.nan)

            outputs.append({
                "compound": compound_name,
                "condition": condition,
                "Top (A)": round(float(A), 4),
                "Slope (B)": round(float(B), 4),
                "EC50 (C, µM)": round(float(C), 6),
                "Bottom (D)": round(float(D), 4),
                "C CI lower (µM)": round(float(C - ci[2]), 6) if np.isfinite(ci[2]) else None,
                "C CI upper (µM)": round(float(C + ci[2]), 6) if np.isfinite(ci[2]) else None
            })

        except Exception as e:
            outputs.append({
                "compound": compound_name,
                "condition": condition,
                "Top (A)": None,
                "Slope (B)": None,
                "EC50 (C, µM)": None,
                "Bottom (D)": None,
                "C CI lower (µM)": None,
                "C CI upper (µM)": None,
                "error": str(e)
            })

results_df = pd.DataFrame(outputs)
out_path = OUT_DIR / "logistic_fit_results.csv"
results_df.to_csv(out_path, index=False)

print(f"Saved results to '{out_path.as_posix()}'")

Saved results to 'outputs/affinity/logistic_fit_results.csv'
